# Download the dataset & get the packeges

In [32]:
!pip install transformers datasets evaluate
from IPython.display import clear_output
clear_output()

In [4]:
!wget https://noisy-text.github.io/2017/files/wnut17train.conll
!wget https://noisy-text.github.io/2017/files/emerging.dev.conll
!wget https://noisy-text.github.io/2017/files/emerging.test.annotated
clear_output()

# Transofrm the data and convert to 🤗 Dataset

In [5]:
from pathlib import Path
import re

def read_wnut(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            if len(line.split('\t'))  < 2:
                continue
            # print(line, len(line.split('\t')))
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)
    return token_docs, tag_docs



In [6]:
train_tokens, train_tags = read_wnut('wnut17train.conll')
val_tokens, val_tags = read_wnut('emerging.dev.conll')
test_tokens, test_tags = read_wnut('emerging.test.annotated')

In [7]:
total_tags = train_tags + val_tags + test_tags

unique_tags = set(tag for doc in total_tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

train_tag_id = [[tag2id[tag] for tag in doc] for doc in train_tags]
val_tag_id = [[tag2id[tag] for tag in doc] for doc in val_tags]
test_tag_id = [[tag2id[tag] for tag in doc] for doc in test_tags]

In [8]:
from datasets import  Dataset, DatasetDict

train_data = {
    'tokens': train_tokens,
    'tag': train_tags,
    'tag_id': train_tag_id,
}

test_data = {
    'tokens': test_tokens,
    'tag': test_tags,
    'tag_id': test_tag_id,
}

validation_data = {
    'tokens': val_tokens,
    'tag': val_tags,
    'tag_id': val_tag_id,
}

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(validation_data)
test_dataset = Dataset.from_dict(test_data)

whole_dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

In [9]:
whole_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tag', 'tag_id'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['tokens', 'tag', 'tag_id'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['tokens', 'tag', 'tag_id'],
        num_rows: 1287
    })
})

# Let the fun begin

In [10]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
tokenizer.is_fast

True

In [12]:
inputs = tokenizer(whole_dataset["train"][0]["tokens"], is_split_into_words=True)
print(inputs.tokens())

['[CLS]', '@', 'p', '##aul', '##walk', 'It', "'", 's', 'the', 'view', 'from', 'where', 'I', "'", 'm', 'living', 'for', 'two', 'weeks', '.', 'Empire', 'State', 'Building', '=', 'E', '##SB', '.', 'Pretty', 'bad', 'storm', 'here', 'last', 'evening', '.', '[SEP]']


In [13]:
print(inputs.word_ids())

[None, 0, 0, 0, 0, 1, 2, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, None]


In [14]:
# labels in int
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [15]:
labels = whole_dataset["train"][0]["tag_id"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 6, 5, 5, 10, 6, 10, 10, 10, 10, 10, 10, 10, 10]
[-100, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 6, 5, 5, 10, 6, 6, 10, 10, 10, 10, 10, 10, 10, 10, -100]


In [16]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["tag_id"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [17]:
tokenized_datasets = whole_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=whole_dataset["train"].column_names,
)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [18]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, return_tensors="tf"
)

In [19]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


<tf.Tensor: shape=(2, 35), dtype=int64, numpy=
array([[-100,   10,   10,   10,   10,   10,   10,   10,   10,   10,   10,
          10,   10,   10,   10,   10,   10,   10,   10,   10,    6,    5,
           5,   10,    6,    6,   10,   10,   10,   10,   10,   10,   10,
          10, -100],
       [-100,   10,   10,   10,   10,   10,   10,    4,    4,    4,   10,
          10,   10,   10,   10,   10,   10,   10,   10,   10,   10,   10,
          10,   10,   10,   10,   10,   10,   10,   10,   10,   10,   10,
        -100, -100]])>

In [20]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 6, 5, 5, 10, 6, 6, 10, 10, 10, 10, 10, 10, 10, 10, -100]
[-100, 10, 10, 10, 10, 10, 10, 4, 4, 4, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, -100]


In [21]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)

tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

In [22]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2tag,
    label2id=tag2id,
)

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
model.config.num_labels

13

In [25]:
from huggingface_hub import notebook_login

notebook_login()

In [26]:
from transformers import create_optimizer
import tensorflow as tf

# Train in mixed-precision float16
# Comment this line out if you're using a GPU that will not benefit from this
# tf.keras.mixed_precision.set_global_policy("mixed_float16")

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_epochs = 15
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

In [27]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(output_dir="bert-finetuned-ner-noval", tokenizer=tokenizer)

model.fit(
    tf_train_dataset,
    # validation_data=tf_eval_dataset,
    callbacks=[callback],
    epochs=num_epochs,
)

Cloning https://huggingface.co/J1mb0o/bert-finetuned-ner-noval into local empty directory.


Epoch 1/15
213/213 [==============================] - 86s 286ms/step - loss: 0.2988
Epoch 2/15
213/213 [==============================] - 81s 382ms/step - loss: 0.1522
Epoch 3/15
213/213 [==============================] - 87s 409ms/step - loss: 0.0945
Epoch 4/15
213/213 [==============================] - 82s 384ms/step - loss: 0.0642
Epoch 5/15
213/213 [==============================] - 88s 415ms/step - loss: 0.0421
Epoch 6/15
213/213 [==============================] - 82s 383ms/step - loss: 0.0286
Epoch 7/15
213/213 [==============================] - 89s 417ms/step - loss: 0.0207
Epoch 8/15
213/213 [==============================] - 88s 414ms/step - loss: 0.0149
Epoch 9/15
213/213 [==============================] - 88s 414ms/step - loss: 0.0120
Epoch 10/15
213/213 [==============================] - 79s 372ms/step - loss: 0.0103
Epoch 11/15
213/213 [==============================] - 87s 411ms/step - loss: 0.0085
Epoch 12/15
213/213 [==============================] - 87s 411ms/step - lo

In [31]:
!pip install seqeval

In [33]:
import evaluate

metric = evaluate.load("seqeval")

In [38]:
whole_dataset["train"]

Dataset({
    features: ['tokens', 'tag', 'tag_id'],
    num_rows: 3394
})

In [44]:
unique_tags

{'B-corporation',
 'B-creative-work',
 'B-group',
 'B-location',
 'B-person',
 'B-product',
 'I-corporation',
 'I-creative-work',
 'I-group',
 'I-location',
 'I-person',
 'I-product',
 'O'}

In [45]:
label_names = list(unique_tags)

In [51]:
whole_dataset["train"][1]["tag"]

['O',
 'O',
 'O',
 'O',
 'B-group',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [48]:
labels = whole_dataset["train"][0]["tag_id"]
labels = [label_names[i] for i in labels]
print(labels)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [49]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'location': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [52]:
import numpy as np

all_predictions = []
all_labels = []
for batch in tf_eval_dataset:
    logits = model.predict_on_batch(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    for prediction, label in zip(predictions, labels):
        for predicted_idx, label_idx in zip(prediction, label):
            if label_idx == -100:
                continue
            all_predictions.append(label_names[predicted_idx])
            all_labels.append(label_names[label_idx])
metric.compute(predictions=[all_predictions], references=[all_labels])

{'corporation': {'precision': 0.16666666666666666,
  'recall': 0.2,
  'f1': 0.1818181818181818,
  'number': 80},
 'creative-work': {'precision': 0.41237113402061853,
  'recall': 0.17937219730941703,
  'f1': 0.25,
  'number': 223},
 'group': {'precision': 0.21666666666666667,
  'recall': 0.16455696202531644,
  'f1': 0.18705035971223022,
  'number': 79},
 'location': {'precision': 0.6341463414634146,
  'recall': 0.5342465753424658,
  'f1': 0.5799256505576209,
  'number': 146},
 'person': {'precision': 0.7734138972809668,
  'recall': 0.5272914521112255,
  'f1': 0.6270667483159829,
  'number': 971},
 'product': {'precision': 0.3695652173913043,
  'recall': 0.2361111111111111,
  'f1': 0.28813559322033905,
  'number': 144},
 'overall_precision': 0.6132743362831858,
 'overall_recall': 0.4217894096165551,
 'overall_f1': 0.4998196898665705,
 'overall_accuracy': 0.9259298902863259}